In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
# import tensorflow-io as tfio
import warnings
warnings.filterwarnings('ignore') 

import gctf
import os
import random 
import PIL
from PIL import Image

from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.applications import EfficientNetB0,EfficientNetB3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Flatten,GlobalAveragePooling2D,BatchNormalization, Activation

from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.losses import BinaryCrossentropy 
from tensorflow.keras.metrics import AUC

In [ ]:
#class Config is for efficient b3 512 input shape
class config:
    seed=42
    batch_size=4
    IMG_SIZE=512
    IMG_SHAPE=(IMG_SIZE,IMG_SIZE,3)
    dropout_rate=0.2
    num_classes=1
    AUTOTUNE=tf.data.experimental.AUTOTUNE
    N_SPLITS=5
    learning_rate=1e-4
    epochs=30

#class Config1 is for efficient b5 456 input shape

class config1:
    seed=42
    batch_size=4
    IMG_SIZE=456
    IMG_SHAPE=(IMG_SIZE,IMG_SIZE,3)
    dropout_rate=0.4
    num_classes=1
    AUTOTUNE=tf.data.experimental.AUTOTUNE
    N_SPLITS=5
    learning_rate=1e-5
    epochs=15
    

In [ ]:
def seed_all(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONASHSEED']=str(seed)
    os.environ['TF_DETERMINISTIC_OPS']='1'

seed_all(config.seed)

In [ ]:
test_data=pd.read_csv('data/Test.csv')
sub=pd.read_csv('data/SampleSubmission.csv')

In [ ]:
training_data='/media/revanth/01D7A0158DB621C0/competitions/zindi/weekend_hackathon_road_segmentation/'
# train_data['file_path']=training_data+train_data['Image_ID']+'.jpeg'
test_data['file_path']=training_data+test_data['Image_ID']+'.jpeg'

In [ ]:
def process_valid_data(image_path):
    image=tf.io.read_file(image_path)
    image=tf.io.decode_jpeg(image,channels=3)
    image = tf.image.random_brightness(image, 0.3)
    image = tf.image.random_flip_left_right(image, seed=None)
    image = tf.image.random_flip_up_down(image)
    image=tf.image.resize(image,size=[config.IMG_SIZE,config.IMG_SIZE])
    image=tf.cast(image,dtype='float32')/255.0
    return image
def process_valid_data1(image_path):
    image=tf.io.read_file(image_path)
    image=tf.io.decode_jpeg(image,channels=3)
    image=tf.image.resize(image,size=[config1.IMG_SIZE,config1.IMG_SIZE])
    image=tf.cast(image,dtype='float32')/255.0
    return image



In [ ]:
#processing efficientnet b3
val_ds = tf.data.Dataset.from_tensor_slices((test_data.file_path.values)).map(process_valid_data , num_parallel_calls=16).batch(config.batch_size).prefetch(config.AUTOTUNE)

#Processing efficientnet b5
val_ds1 = tf.data.Dataset.from_tensor_slices((test_data.file_path.values)).map(process_valid_data1 , num_parallel_calls=16).batch(config1.batch_size).prefetch(config1.AUTOTUNE)

In [ ]:
#Efficientnet b3 models
model_list=['models/best_model_0_.hdf5',
'models/best_model_1_.hdf5', 
'models/best_model_2_.hdf5',
'models/best_model_3_.hdf5',
'models/best_model_4_.hdf5']

In [ ]:
#Efficientnet b5 models
model_list_2=['models/bestb0_384_model_0_.hdf5',
'models/bestb0_384_model_1_.hdf5' ,
'models/bestb0_384_model_2_.hdf5']

In [ ]:
from tensorflow.keras.models import load_model

preds=[]
preds1=[]

for i in model_list:
    print(i)
    model=load_model(i)
    preds_test=model.predict(val_ds)
    preds.append(preds_test)
    tf.keras.backend.clear_session()

for i in model_list_2:
    print(i)
    model=load_model(i)
    
    preds_test1=model.predict(val_ds1)
    preds1.append(preds_test1)
    tf.keras.backend.clear_session()

In [ ]:
preds

In [ ]:
sub=pd.read_csv('data/SampleSubmission.csv')


In [ ]:
sub.head()

In [ ]:
# sub['Target']=(preds[0]+preds[1]+preds[2]+preds[3]+preds[4])/5
sub['Target']=(preds[0]**4+preds[1]**4+preds[2]**4+preds[3]**4+preds[4]**4+preds1[0]**4+preds1[1]**4+preds1[2]**4)/8

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submissions/8_model_average-power-4-ensemble-fold-avg-b5-456-cv-0.0.95',index=False)